In [29]:
import plotly.express as px
from utils.ops import load_yaml
from  pathlib import Path
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [30]:
site = 1   
site_label  = 'AM'

In [31]:
cfg = load_yaml('cfg.yaml')
site_cfg = load_yaml(f'site_{site}.yaml')

preparation_params = cfg['preparation_params']
paths_params = cfg['paths']
general_params = cfg['general_params']
experiments_paths = general_params['experiments_folders']

images_path = Path('images')
images_path.mkdir(exist_ok=True)

In [32]:
def results(experiments):
    exp_results = []
    for exp_i in experiments:
        #experiment_params = cfg['experiments'][f'exp_{experiments[exp_i]}']
        exp_path = Path(paths_params['experiments']) / f'exp_{exp_i}'
        experiment_params = cfg['experiments'][f'exp_{exp_i}']

        results_path = exp_path / experiments_paths['results']
        final_results_file = results_path / f'results_{exp_i}.data'

        result_df = pd.read_pickle(final_results_file)
        exp_results.append(
            {
                'name': f'{experiment_params["short"]}', # ({exp_i})',
                'Cloud Condition': 'Global',
                'metric': 'Precision',
                'value': result_df['global_precision'][0],
                'group_model': experiment_params['group_model']
            }
        )
        exp_results.append(
            {
                'name': f'{experiment_params["short"]}', # ({exp_i})',
                'Cloud Condition': 'Global',
                'metric': 'Recall',
                'value': result_df['global_recall'][0],
                'group_model': experiment_params['group_model']
            }
        )
        exp_results.append(
            {
                'name': f'{experiment_params["short"]}', # ({exp_i})',
                'Cloud Condition': 'Global',
                'metric': 'F1-Score',
                'value': result_df['global_f1'][0],
                'group_model': experiment_params['group_model']
            }
        )
        exp_results.append(
            {
                'name': f'{experiment_params["short"]}', # ({exp_i})',
                'Cloud Condition': 'No Cloud',
                'metric': 'Precision',
                'value': result_df['no_cloud_precision'][0],
                'group_model': experiment_params['group_model']
            }
        )
        exp_results.append(
            {
                'name': f'{experiment_params["short"]}', # ({exp_i})',
                'Cloud Condition': 'No Cloud',
                'metric': 'Recall',
                'value': result_df['no_cloud_recall'][0],
                'group_model': experiment_params['group_model']
            }
        )
        exp_results.append(
            {
                'name': f'{experiment_params["short"]}', # ({exp_i})',
                'Cloud Condition': 'No Cloud',
                'metric': 'F1-Score',
                'value': result_df['no_cloud_f1'][0],
                'group_model': experiment_params['group_model']
            }
        )
        exp_results.append(
            {
                'name': f'{experiment_params["short"]}', # ({exp_i})',
                'Cloud Condition': 'Cloudy',
                'metric': 'Precision',
                'value': result_df['cloud_precision'][0],
                'group_model': experiment_params['group_model']
            }
        )
        exp_results.append(
            {
                'name': f'{experiment_params["short"]}', # ({exp_i})',
                'Cloud Condition': 'Cloudy',
                'metric': 'Recall',
                'value': result_df['cloud_recall'][0],
                'group_model': experiment_params['group_model']
            }
        )
        exp_results.append(
            {
                'name': f'{experiment_params["short"]}', # ({exp_i})',
                'Cloud Condition': 'Cloudy',
                'metric': 'F1-Score',
                'value': result_df['cloud_f1'][0],
                'group_model': experiment_params['group_model']
            }
        )
        
        
    return pd.DataFrame(exp_results)

## Results

In [33]:
experiments = [
    6,
    11,
    12,
    13,    
    2,
    1,
    3,
    4,
    5,
    7,
    8,
    56,
    61,
    62,
    63,
    52,
    51,
    53,
    54,
    55,
    57,
    58
]
pixel_conds = ['No Cloud', 'Cloudy', 'Global']
metrics = ['Recall', 'Precision', 'F1-Score']
exp_results = results(experiments)

In [34]:
pixel_cond = pixel_conds[2]
for metric in metrics:
    filt_res = exp_results[(exp_results['Cloud Condition'] == pixel_cond) & (exp_results['metric'] == metric)]
    opt_cloud_free_results = filt_res[(filt_res['name'].str.contains('Cloud-free') )]
    #filt_res = filt_res[~(filt_res['name'].str.contains('Cloud-free') )]

    fig = make_subplots(rows=1, cols = 2, subplot_titles=('CNN', 'SWIN-Unet'), shared_yaxes = True)

    cnn_results = filt_res[filt_res['group_model'] == 'CNN']
    cnn_opt_cloud_free_results =  opt_cloud_free_results[opt_cloud_free_results['group_model'] == 'CNN']

    for i, name_res in enumerate(cnn_results['name']):
        loc_res = cnn_results[cnn_results['name'] == name_res]
        fig.add_bar( x = loc_res['name'], y=loc_res['value'], name = loc_res['name'].iloc[0], text=loc_res['value'], texttemplate='%{text:.2f}', textposition='inside', row=1, col=1, marker_color = px.colors.qualitative.Alphabet[i])

    fig.add_hline(
        y = cnn_opt_cloud_free_results['value'].max(), #cnn_opt_cloud_free_results['value'].iloc[0], 
        row=1, 
        col=1, 
        line_dash = 'dot',
        line_color = '#000000' #px.colors.qualitative.Alphabet[0],
        )

    swin_results = filt_res[filt_res['group_model'] == 'SWIN-Unet']
    swin_opt_cloud_free_results =  opt_cloud_free_results[opt_cloud_free_results['group_model'] == 'SWIN-Unet']
    for i, name_res in enumerate(swin_results['name']):
        loc_res = swin_results[swin_results['name'] == name_res]
        fig.add_bar( x = loc_res['name'], y=loc_res['value'], name = loc_res['name'].iloc[0], showlegend = False, text=loc_res['value'], texttemplate='%{text:.2f}', textposition='inside', row=1, col=2, marker_color = px.colors.qualitative.Alphabet[i])

    fig.add_hline(
        y = swin_opt_cloud_free_results['value'].max(),  #swin_opt_cloud_free_results['value'].iloc[0], 
        row=1, 
        col=2, 
        line_dash = 'dot',
        line_color = '#000000' #px.colors.qualitative.Alphabet[0],
        )

    fig.update_yaxes(range=[0.05, 1.05], title_text = '', tickformat = '.1f') 
    fig.update_layout(
        width = 1600,
        height = 520,
        legend_title_text = 'Model', 
        legend_font_size = 16,
        #legend_orientation = 'h',
        font_size = 12,
        bargap=0.30,
        title_text = f'{metric} ({pixel_cond}) [Site:{site_label}]',
        title_font_size = 20,
        title_x = 0.5
        )
    fig.show()
    fig.write_html(images_path/f'site_{site}_{pixel_cond}_{metric}.html')
    fig.write_image(images_path/f'site_{site}_{pixel_cond}_{metric}.png')

In [35]:
pixel_cond = pixel_conds[0]
for metric in metrics:
    filt_res = exp_results[(exp_results['Cloud Condition'] == pixel_cond) & (exp_results['metric'] == metric)]
    opt_cloud_free_results = filt_res[(filt_res['name'].str.contains('Cloud-free') )]
    #filt_res = filt_res[~(filt_res['name'].str.contains('Cloud-free') )]

    fig = make_subplots(rows=1, cols = 2, subplot_titles=('CNN', 'SWIN-Unet'), shared_yaxes = True)

    cnn_results = filt_res[filt_res['group_model'] == 'CNN']
    cnn_opt_cloud_free_results =  opt_cloud_free_results[opt_cloud_free_results['group_model'] == 'CNN']

    for i, name_res in enumerate(cnn_results['name']):
        loc_res = cnn_results[cnn_results['name'] == name_res]
        fig.add_bar( x = loc_res['name'], y=loc_res['value'], name = loc_res['name'].iloc[0], text=loc_res['value'], texttemplate='%{text:.2f}', textposition='inside', row=1, col=1, marker_color = px.colors.qualitative.Alphabet[i])

    fig.add_hline(
        y = cnn_opt_cloud_free_results['value'].max(), #.iloc[0], 
        row=1, 
        col=1, 
        line_dash = 'dot',
        line_color = '#000000' #px.colors.qualitative.Alphabet[0],
        )

    swin_results = filt_res[filt_res['group_model'] == 'SWIN-Unet']
    swin_opt_cloud_free_results =  opt_cloud_free_results[opt_cloud_free_results['group_model'] == 'SWIN-Unet']
    for i, name_res in enumerate(swin_results['name']):
        loc_res = swin_results[swin_results['name'] == name_res]
        fig.add_bar( x = loc_res['name'], y=loc_res['value'], name = loc_res['name'].iloc[0], showlegend = False, text=loc_res['value'], texttemplate='%{text:.2f}', textposition='inside', row=1, col=2, marker_color = px.colors.qualitative.Alphabet[i])

    fig.add_hline(
        y = swin_opt_cloud_free_results['value'].max(), #.iloc[0], 
        row=1, 
        col=2, 
        line_dash = 'dot',
        line_color = '#000000' # px.colors.qualitative.Alphabet[0],
        )

    fig.update_yaxes(range=[0.05, 1.05], title_text = '', tickformat = '.1f') 
    fig.update_layout(
        width = 1600,
        height = 520,
        legend_title_text = 'Model', 
        legend_font_size = 16,
        #legend_orientation = 'h',
        font_size = 12,
        bargap=0.30,
        title_text = f'{metric} ({pixel_cond}) [Site:{site_label}]',
        title_font_size = 20,
        title_x = 0.5
        )
    fig.show()
    fig.write_html(images_path/f'site_{site}_{pixel_cond}_{metric}.html')
    fig.write_image(images_path/f'site_{site}_{pixel_cond}_{metric}.png')

In [36]:
pixel_cond = pixel_conds[1]
for metric in metrics:
    filt_res = exp_results[(exp_results['Cloud Condition'] == pixel_cond) & (exp_results['metric'] == metric)]
    sar_results = filt_res[(filt_res['name'].str.contains('SAR') )]
    filt_res = filt_res[~(filt_res['name'].str.contains('Cloud-free') )]
    #filt_res = filt_res[~(filt_res['name'].str.contains('SAR') )]

    fig = make_subplots(rows=1, cols = 2, subplot_titles=('CNN', 'SWIN-Unet'), shared_yaxes = True)

    cnn_results = filt_res[filt_res['group_model'] == 'CNN']
    cnn_sar_results =  sar_results[sar_results['group_model'] == 'CNN']

    for i, name_res in enumerate(cnn_results['name']):
        loc_res = cnn_results[cnn_results['name'] == name_res]
        fig.add_bar( x = loc_res['name'], y=loc_res['value'], name = loc_res['name'].iloc[0], text=loc_res['value'], texttemplate='%{text:.2f}', textposition='inside', row=1, col=1, marker_color = px.colors.qualitative.Alphabet[i+4])

    fig.add_hline(
        y = cnn_sar_results['value'].iloc[0], 
        row=1, 
        col=1, 
        line_dash = 'dot',
        line_color = px.colors.qualitative.Alphabet[4],
        )

    swin_results = filt_res[filt_res['group_model'] == 'SWIN-Unet']
    swin_sar_results =  sar_results[sar_results['group_model'] == 'SWIN-Unet']
    for i, name_res in enumerate(swin_results['name']):
        loc_res = swin_results[swin_results['name'] == name_res]
        fig.add_bar( x = loc_res['name'], y=loc_res['value'], name = loc_res['name'].iloc[0], showlegend = False, text=loc_res['value'], texttemplate='%{text:.2f}', textposition='inside', row=1, col=2, marker_color = px.colors.qualitative.Alphabet[i+4])

    fig.add_hline(
        y = swin_sar_results['value'].iloc[0], 
        row=1, 
        col=2, 
        line_dash = 'dot',
        line_color = px.colors.qualitative.Alphabet[4],
        )

    fig.update_yaxes(range=[0.05, 1.05], title_text = '', tickformat = '.1f') 
    fig.update_layout(
        width = 1600,
        height = 520,
        legend_title_text = 'Model', 
        legend_font_size = 16,
        #legend_orientation = 'h',
        font_size = 12,
        bargap=0.30,
        title_text = f'{metric} ({pixel_cond}) [Site:{site_label}]',
        title_font_size = 20,
        title_x = 0.5
        )
    fig.show()
    fig.write_html(images_path/f'site_{site}_{pixel_cond}_{metric}.html')
    fig.write_image(images_path/f'site_{site}_{pixel_cond}_{metric}.png')

## No Cloud Results

In [37]:
experiments = [
    6,
    2,
    9,
    10,
    11,
    12,
    13,
    14,
    15,
    16,
    17,
    56,
    52,
    59,
    60,
    61,
    62,
    63,
    64,
    65,
    66,
    67
]
pixel_conds = ['No Cloud', 'Cloudy', 'Global']
metrics = ['Recall', 'Precision', 'F1-Score']
exp_results = results(experiments)

In [38]:
pixel_cond = pixel_conds[2]
for metric in metrics:
    filt_res = exp_results[(exp_results['Cloud Condition'] == pixel_cond) & (exp_results['metric'] == metric)]
    opt_cloud_free_results = filt_res[(filt_res['name'].str.contains('Cloud-free') )]
    #filt_res = filt_res[~(filt_res['name'].str.contains('Cloud-free') )]

    fig = make_subplots(rows=1, cols = 2, subplot_titles=('CNN', 'SWIN-Unet'), shared_yaxes = True)

    cnn_results = filt_res[filt_res['group_model'] == 'CNN']
    cnn_opt_cloud_free_results =  opt_cloud_free_results[opt_cloud_free_results['group_model'] == 'CNN']

    for i, name_res in enumerate(cnn_results['name']):
        loc_res = cnn_results[cnn_results['name'] == name_res]
        fig.add_bar( x = loc_res['name'], y=loc_res['value'], name = loc_res['name'].iloc[0], text=loc_res['value'], texttemplate='%{text:.2f}', textposition='inside', row=1, col=1, marker_color = px.colors.qualitative.Alphabet[i])

    fig.add_hline(
        y = cnn_opt_cloud_free_results['value'].max(), #cnn_opt_cloud_free_results['value'].iloc[0], 
        row=1, 
        col=1, 
        line_dash = 'dot',
        line_color = '#000000' #px.colors.qualitative.Alphabet[0],
        )

    swin_results = filt_res[filt_res['group_model'] == 'SWIN-Unet']
    swin_opt_cloud_free_results =  opt_cloud_free_results[opt_cloud_free_results['group_model'] == 'SWIN-Unet']
    for i, name_res in enumerate(swin_results['name']):
        loc_res = swin_results[swin_results['name'] == name_res]
        fig.add_bar( x = loc_res['name'], y=loc_res['value'], name = loc_res['name'].iloc[0], showlegend = False, text=loc_res['value'], texttemplate='%{text:.2f}', textposition='inside', row=1, col=2, marker_color = px.colors.qualitative.Alphabet[i])

    fig.add_hline(
        y = swin_opt_cloud_free_results['value'].max(),  #swin_opt_cloud_free_results['value'].iloc[0], 
        row=1, 
        col=2, 
        line_dash = 'dot',
        line_color = '#000000' #px.colors.qualitative.Alphabet[0],
        )

    fig.update_yaxes(range=[0.05, 1.05], title_text = '', tickformat = '.1f') 
    fig.update_layout(
        width = 1600,
        height = 520,
        legend_title_text = 'Model', 
        legend_font_size = 16,
        #legend_orientation = 'h',
        font_size = 12,
        bargap=0.30,
        title_text = f'{metric} ({pixel_cond}) [Site:{site_label}]',
        title_font_size = 20,
        title_x = 0.5
        )
    fig.show()
    fig.write_html(images_path/f'site_{site}_{pixel_cond}_{metric}.html')
    fig.write_image(images_path/f'site_{site}_{pixel_cond}_{metric}.png')